In [1]:
from dotenv import load_dotenv

load_dotenv()

True

### Using Language Models

In [2]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model='gpt-4o-mini')

In [3]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content='Translate the following from English into Korean'),
    HumanMessage(content='Hi!')
]

model.invoke(messages)

AIMessage(content='안녕하세요!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'stop', 'logprobs': None}, id='run-43ecb9c7-a9a4-43eb-af3f-602946346376-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### OutputParsers

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [5]:
result = model.invoke(messages)

In [6]:
parser.invoke(result)

'안녕하세요!'

### Prompt Templates

In [8]:
from langchain_core.prompts import ChatPromptTemplate

system_template = "Translate the following into {language}:"
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),
     ("user", "{text}")
    ]
)

result = prompt_template.invoke({'language': 'korean', 'text': 'Hi!'})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into korean:', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={})])

In [9]:
result.to_messages()

[SystemMessage(content='Translate the following into korean:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi!', additional_kwargs={}, response_metadata={})]

### Vector stores and retrievers

In [10]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [11]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings()
)

In [12]:
vectorstore.similarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [14]:
await vectorstore.asimilarity_search("cat")

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

In [16]:
vectorstore.similarity_search_with_score("cat")

[(Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.37521976232528687),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.48278287053108215),
 (Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  0.49597978591918945),
 (Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.'),
  0.4974355101585388)]

In [17]:
embedding = OpenAIEmbeddings()

vectorstore.similarity_search_by_vector(embedding.embed_query("cat"))

[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
 Document(metadata={'source': 'bird-pets-doc'}, page_content='Parrots are intelligent birds capable of mimicking human speech.')]

### Retrievers

In [20]:
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

In [22]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 1},
)

retriever.batch(["cat", "shark"])

[[Document(metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')],
 [Document(metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.')]]

In [23]:
from langchain_core.runnables import RunnablePassthrough

llm = ChatOpenAI(model='gpt-4o-mini')

message = """
Answer this question using the provided context only.

Question:
{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([("human", message)])

rag_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | llm | StrOutputParser()

In [24]:
rag_chain.invoke("Tell me about cats")

'Cats are independent pets that often enjoy their own space.'

### Build an Agent

In [25]:
from langchain_community.tools.tavily_search import TavilySearchResults

search_tool = TavilySearchResults(max_results=2)
search_results = search_tool.invoke("What is the weather in Incheon?")

print(search_results)

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Incheon', 'region': '', 'country': 'South Korea', 'lat': 37.4536, 'lon': 126.7317, 'tz_id': 'Asia/Seoul', 'localtime_epoch': 1734567739, 'localtime': '2024-12-19 09:22'}, 'current': {'last_updated_epoch': 1734567300, 'last_updated': '2024-12-19 09:15', 'temp_c': -1.6, 'temp_f': 29.2, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.9, 'wind_kph': 4.7, 'wind_degree': 25, 'wind_dir': 'NNE', 'pressure_mb': 1031.0, 'pressure_in': 30.45, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 32, 'cloud': 0, 'feelslike_c': -3.2, 'feelslike_f': 26.2, 'windchill_c': -3.2, 'windchill_f': 26.2, 'heatindex_c': -1.6, 'heatindex_f': 29.2, 'dewpoint_c': -16.0, 'dewpoint_f': 3.3, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.3, 'gust_mph': 3.7, 'gust_kph': 5.9}}"}, {'url': 'https://world-weather.info/forecast/south_korea/incheon/december-2024/', 'content': 

In [26]:
llm_with_tools = llm.bind_tools(tools=[search_tool])

response = llm_with_tools.invoke([HumanMessage(content="Hi~")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

response = llm_with_tools.invoke([HumanMessage(content="What is the weather in Incheon?")])
print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: Hello! How can I assist you today?
ToolCalls: []
ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Incheon'}, 'id': 'call_Ui2qLo857iKTbcEku0WE2EGn', 'type': 'tool_call'}]


### Create the agent

In [27]:
# create_react_agent will call .bind_tools for us under the hood.
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(llm, [search_tool])

In [32]:
response = agent_executor.invoke({"messages": [HumanMessage(content="hi")]})

response["messages"]

[HumanMessage(content='hi', additional_kwargs={}, response_metadata={}, id='f4185a21-a11d-4905-9f58-71b098b3428e'),
 AIMessage(content='Hello! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 80, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-f3978fd6-e463-4289-8df4-2474d92fe89d-0', usage_metadata={'input_tokens': 80, 'output_tokens': 10, 'total_tokens': 90, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [33]:
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}, id='b5255b28-0b7f-44fb-8333-47c2142ebce8'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_IaHSJjEKeIPBoKUKlSv4UHV3', 'function': {'arguments': '{"query":"current weather San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 86, 'total_tokens': 107, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_6fc10e10eb', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4e4dc95c-f203-47cb-b7a2-2757fd17760d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather San Francisco'}, 'id': 'call_IaHSJjEKeIPBoK

### Streaming tokens

In [39]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="Whats the weather in Seoul?")]}, version="v1"
):
    kind = event['event']
    if kind == "on_chain_start":
        if event['name'] == 'Agent':
            print(f"Starting agent: {event['name']} with input: {event['data'].get('input')}")
    elif kind == "on_chain_end":
        if event['name'] == 'Agent':
            print()
            print("--")
            print(f"Done agent: {event['name']} with input: {event['data'].get('output')['output']}")
            
    if kind == "on_chat_model_stream":
        content = event['data']['chunk'].content
        if content:
            print(content, end="|")
            
    elif kind == "on_tool_start":
        print("--")
        print(f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}")
        
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'Seoul weather'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'Seoul\', \'region\': \'\', \'country\': \'South Korea\', \'lat\': 37.5664, \'lon\': 126.9997, \'tz_id\': \'Asia/Seoul\', \'localtime_epoch\': 1734568755, \'localtime\': \'2024-12-19 09:39\'}, \'current\': {\'last_updated_epoch\': 1734568200, \'last_updated\': \'2024-12-19 09:30\', \'temp_c\': -2.0, \'temp_f\': 28.4, \'is_day\': 1, \'condition\': {\'text\': \'Sunny\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/113.png\', \'code\': 1000}, \'wind_mph\': 2.2, \'wind_kph\': 3.6, \'wind_degree\': 27, \'wind_dir\': \'NNE\', \'pressure_mb\': 1031.0, \'pressure_in\': 30.45, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 32, \'cloud\': 0, \'feelslike_c\': -2.0, \'feelslike_f\': 28.4, \'windchill_c\': -2.0, \'windchill_f\': 28.4, \'heatindex_c\': -2.0, \'he

### Adding in memory

In [49]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [50]:
agent_executor = create_react_agent(llm, [search_tool], checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [52]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Hi! I'm jiyun")]}, config
):
    print(chunk)
    print("-----")

{'agent': {'messages': [AIMessage(content='Hi Jiyun! How can I assist you today?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 13, 'prompt_tokens': 85, 'total_tokens': 98, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_39a40c96a0', 'finish_reason': 'stop', 'logprobs': None}, id='run-80883df5-992f-4015-9efc-b49887685b70-0', usage_metadata={'input_tokens': 85, 'output_tokens': 13, 'total_tokens': 98, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
-----


In [53]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Jiyun. How can I help you today, Jiyun?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 110, 'total_tokens': 129, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-5e9f085a-44dc-4001-ab29-46f89a491123-0', usage_metadata={'input_tokens': 110, 'output_tokens': 19, 'total_tokens': 129, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----


In [54]:
config = {"configurable": {"thread_id": "xyz123"}}

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't have access to personal data about users unless it has been shared with me during our conversation. Therefore, I don't know your name. If you'd like to share it, feel free to do so!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 84, 'total_tokens': 126, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_reason': 'stop', 'logprobs': None}, id='run-bd6b7f6c-1bde-46d7-882a-7cb70b4280e6-0', usage_metadata={'input_tokens': 84, 'output_tokens': 42, 'total_tokens': 126, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]}}
----
